In [1]:
import numpy as np
import matplotlib as mpl

In [2]:
# Question 1
#: the diameter of the wire (x0), 
#the mean of the diameter of coil(x1)
#and the number of active coils (x2)

In [3]:
arr_X = np.array([1, 2, 3])

In [4]:
#implementation of f(x)

In [5]:
def f(x, fCount=[0]):
    fun = int(x[0] ** 2)*x[1]*(2 + x[2])
    fCount[0]+=1
    return [fun, fCount[0]]

In [6]:
print(f(arr_X))
print(f(arr_X))
print(f(arr_X))

[10, 1]
[10, 2]
[10, 3]


In [7]:
#implementation of g1(x)

In [8]:
def g1(x, g1Count=[0]):
    g1Count[0]+=1
    fun = float(1 - (((x[1]**3)*x[2])/(71785*(x[0]**4))))
    if ( fun <= 0) :
        return [True, g1Count[0]]
    else:
        return [False, g1Count[0]]
    
    

In [9]:
print (g1(arr_X))
print (g1(arr_X))
print (g1(arr_X))

[False, 1]
[False, 2]
[False, 3]


In [10]:
#implementation of g2(x)

In [11]:
def g2(x, g2Count=[0]):
    fun = (   (((4*x[1]**2) - (x[0]*x[1]))/(12566*((x[1]*(x[0]**3))-(x[0]**4)))) + (1/(5108*(x[0]**2)))  - 1 )
    g2Count[0]+=1
    if fun <= 0:
        return[True, g2Count[0]]
    else:
        return[False, g2Count[0]]

In [12]:
print (g2(arr_X))
print (g2(arr_X))
print (g2(arr_X))

[True, 1]
[True, 2]
[True, 3]


In [13]:
def g3(x, g3Count=[0]):
    fun = (1 - (140.45*x[0])/((x[1]**2)*x[2]))
    g3Count[0]+=1
    if fun <= 0:   
        return[True, g3Count[0]]
    else:
        return[False, g3Count[0]]
  

In [14]:
print (g3(arr_X))
print (g3(arr_X))
print (g3(arr_X))

[True, 1]
[True, 2]
[True, 3]


In [15]:
def g4(x, g4Count=[0]):
    fun = (  ((x[0] + x[1])/1.5)-1  )
    g4Count[0]+=1
    if fun <= 0:   
        return[True, g4Count[0]]
    else:
        return[False, g4Count[0]]

In [16]:
print (g4(arr_X))
print (g4(arr_X))
print (g4(arr_X))

[False, 1]
[False, 2]
[False, 3]


In [17]:
#Question 2

In [18]:
wire_lower = 0.005 #lower bound for diameter of the wire
wire_upper = 2 #upper bound for diameter of the wire

coil_lower = 0.25 #lower bound for diameter of coil
coil_upper = 1.3 #upper bound for diameter of the coil

coil_num_lower = 2 #lower bound for number of active coils
coil_num_upper = 15 #upper bound for number of active coils

In [19]:
def generate_random_array():
    #generate random number 
    x0 = np.random.uniform(low=0.05, high=2) #hightest + 1
    x1 = np.random.uniform(low=0.25, high=1.3) #hightest + 1
    x2 = np.random.uniform(low=2, high=15) #hightest + 1
    
    return np.array([x0, x1, x2])
    

In [20]:
print (g3(arr_X))
print (g3(arr_X))
print (g3(arr_X))

[True, 4]
[True, 5]
[True, 6]


In [21]:
def is_feasible(array):
    feasible = False
    g1_arr=g1(array)
    g2_arr=g2(array)
    g3_arr=g3(array)
    g4_arr=g4(array)

#     print(g1_arr)
#     print(g2_arr)
#     print(g3_arr)
#     print(g4_arr)

    if g1_arr[0] and g2_arr[0] and g3_arr[0] and g4_arr[0] :
        feasible = True
    else:
        feasible = False
    return feasible
    

In [43]:
def random_search(best_fx, best_x):
    
    arr_x = generate_random_array()
    #print(arr_x)
    #print(is_feasible(arr_x))
    if is_feasible(arr_x):
        fx = f(arr_x)
        if fx < best_fx:
            best_x = np.array([arr_x[0], arr_x[1], arr_x[2]])
            print("best replaced")
            
    return best_x

In [45]:
best_x = generate_random_array()
best_fx = f(best_x)

for x in range(0, 3000):
    best_x = random_search(best_fx, best_x)
print(best_x)
print(best_fx)
#print(arr_X)

#myArray = np.array(arrX)
#print(myArray)

# test = np.array([1, 2, 3])
#print (g1(arr_X))
# # #def searching(arr_X, best_X, best_fx):



best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
best replaced
[ 0.063364    0.65453299 13.57687388]
[51.342492683637545, 377]


In [ ]:
#Question 3

y = w1*x0 + w2*x1 + w3*x2
